<center><h1><span style="color:#4d61f4">Unbalanced Classes</span></h1></center>

<div style="color:#242456">There's a problem some of you may have encountered with logistic regression (or decision trees, or other classifiers), where the outcome you're trying to predict has "unbalanced classes" - meaning one outcome is much more common than the other. This means the model "gets lazy" and can end up just always predicting the more common outcome. For example, if you're trying to predict whether a passenger on the Titanic survived the disaster or not, then because the majority did not, your model can achieve quite high accuracy just by always predicting "died".

There are a few different approaches to solving this, and one of the simplest is "undersampling". This is where you randomly remove some of your data from the larger class so that the outcomes are more balanced. So instead of training your model on 500 samples, of which 100 are outcome A and 400 are outcome B, you might train your model on 200 samples, of which 100 are outcome A and 100 are outcome B - that way the model is forced to work harder to distinguish between them.

There are a few different ways to achieve undersampling in Python. If you have an unbalanced classification problem, you can follow this notebook to guide you.</div>

In [32]:
import pandas as pd
import numpy as np # you'll need numpy for this undersampling method
# you might also need other libraries, depending on what else you're doing with your data

In [33]:
# you'll need the titanic.csv dataset to be able to run this notebook
titanic = pd.read_csv('./Data/titanic.csv')
titanic.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [34]:
# look at how many of each outcome there are
titanic.Survived.value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [35]:
# and again as proportions
titanic.Survived.value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

<div style="color:#242456">So there are 549 people who survived (62%), and 342 who died (38%). This isn't very badly unbalanced, so it might be ok in this case to carry on with the full data, but let's see how we can undersample the majority class to get a more balanced classification problem.</div>

<div style="color:#242456">We'll undersample the passengers who died to match the survivors. First we need to separate the two classes out by filtering the dataframe with a mask:</div>

In [36]:
# define a mask to filter the dataframe to just those who died
mask = (titanic.Survived == 0)

# apply the mask to get the filtered dataframe
died = titanic[mask]
survived = titanic[~mask] # ~ means "not", so this filters to everyone not included in the mask

<span style="color:#242456">Now we need to randomly select approximately 342 people from the died dataset, to match the survived dataset. Often when undersampling you don't perfectly match the sizes of the two classes, you keep the larger class a bit bigger (so you don't throw away too much data!), so let's undersample to only 400 people who died. We can use the `np.random.choice` function to randomly choose items from a list, and we use this on the list of indices (row numbers) from the died dataset.</span>

In [37]:
chosen_rows = np.random.choice(died.index, size=400, replace=False) # need to set replace=False so we can't choose the same number twice

<span style="color:#242456">This `chosen_rows` variable is just an array of 400 random row numbers from the died dataset. We can use these row numbers to select the 400 corresponding rows:</span>

In [38]:
# now we have the row numbers, we can use .loc to select those rows from the died dataset
died.loc[chosen_rows]

PassengerId  Survived  Pclass                                      Name  \
436          437         0       3      Ford, Miss. Doolina Margaret "Daisy"   
148          149         0       2  Navratil, Mr. Michel ("Louis M Hoffman")   
433          434         0       3                Kallio, Mr. Nikolai Erland   
456          457         0       1                 Millet, Mr. Francis Davis   
96            97         0       1                 Goldschmidt, Mr. George B   
..           ...       ...     ...                                       ...   
377          378         0       1                 Widener, Mr. Harry Elkins   
551          552         0       2               Sharp, Mr. Percival James R   
451          452         0       3           Hagland, Mr. Ingvald Olai Olsen   
783          784         0       3                    Johnston, Mr. Andrew G   
302          303         0       3           Johnson, Mr. William Cahoone Jr   

        Sex   Age  SibSp  Parch             Ticket      Fare Cabin Embarked  
436  female  21.0      2      2         W./C. 6608   34.3750   NaN        S  
148    male  36.5      0      2             230080   26.0000    F2        S  
433    male  17.0      0      0  STON/O 2. 3101274    7.1250   NaN        S  
456    male  65.0      0      0              13509   26.5500   E38        S  
96     male  71.0      0      0           PC 17754   34.6542    A5        C  
..      ...   ...    ...    ...                ...       ...   ...      ...  
377    male  27.0      0      2             113503  211.5000   C82        C  
551    male  27.0      0      0             244358   26.0000   NaN        S  
451    male   NaN      1      0              65303   19.9667   NaN        S  
783    male   NaN      1      2         W./C. 6607   23.4500   NaN        S  
302    male  19.0      0      0               LINE    0.0000   NaN        S  

[400 rows x 12 columns]

<span style="color:#242456">So now we have 400 randomly chosen passengers who died. We can combine those back with the 342 survivors using `pd.concat` (which is a union - TC2) to get a dataset which is closer to balanced for training our classification model:</span>

In [39]:
# union all survivors together with the 400 randomly chosen people who died
undersampled_data = pd.concat([survived, died.loc[chosen_rows]])

In [40]:
# look at our undersampled data
undersampled_data

PassengerId  Survived  Pclass  \
1              2         1       1   
2              3         1       3   
3              4         1       1   
8              9         1       3   
9             10         1       2   
..           ...       ...     ...   
377          378         0       1   
551          552         0       2   
451          452         0       3   
783          784         0       3   
302          303         0       3   

                                                  Name     Sex   Age  SibSp  \
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)  female  27.0      0   
9                  Nasser, Mrs. Nicholas (Adele Achem)  female  14.0      1   
..                                                 ...     ...   ...    ...   
377                          Widener, Mr. Harry Elkins    male  27.0      0   
551                        Sharp, Mr. Percival James R    male  27.0      0   
451                    Hagland, Mr. Ingvald Olai Olsen    male   NaN      1   
783                             Johnston, Mr. Andrew G    male   NaN      1   
302                    Johnson, Mr. William Cahoone Jr    male  19.0      0   

     Parch            Ticket      Fare Cabin Embarked  
1        0          PC 17599   71.2833   C85        C  
2        0  STON/O2. 3101282    7.9250   NaN        S  
3        0            113803   53.1000  C123        S  
8        2            347742   11.1333   NaN        S  
9        0            237736   30.0708   NaN        C  
..     ...               ...       ...   ...      ...  
377      2            113503  211.5000   C82        C  
551      0            244358   26.0000   NaN        S  
451      0             65303   19.9667   NaN        S  
783      2        W./C. 6607   23.4500   NaN        S  
302      0              LINE    0.0000   NaN        S  

[742 rows x 12 columns]

In [41]:
# let's check the results:
undersampled_data.Survived.value_counts()

0    400
1    342
Name: Survived, dtype: int64

<span style="color:#242456">400 died, and 342 survived, as expected. You can now run a train-test split on `undersampled_data` and build a model just as normal, but the model may have better performance, because the classes are more balanced.</span>

<span style="color:#242456">Of course, undersampling does lose some data, and this can make the model worse. You should never <em>assume</em> you should undersample - always fit a model on a training subset of the whole dataset, and if that model doesn't have good enough performance and your classes are unbalanced, consider undersampling as one method to address the issue.</span>

<h2><span style="color:#4d61f4">Summary</span></h2>

<span style="color:#242456">Here's all the key steps collected into one place, as a summary:</span>

In [19]:
# define a mask to filter the data based on the outcome that's unbalanced
mask = (titanic.Survived == 0)

# filter your data to those satisfying the mask and those not - i.e., split it into two groups based on the outcome
died = titanic[mask]
survived = titanic[~mask]

# randomly choose an appropriate number of row numbers from the larger class
chosen_rows = np.random.choice(died.index, size=400, replace=False)

# use those rows to select a random subset of the larger class, and union (concatenate) it back together with the smaller class
undersampled = pd.concat([survived, died.loc[chosen_rows]])

# then do your analysis as you normally would, but using undersampled as your dataset instead of the whole thing

<div style="color:#242456">As well as undersampling, there are other methods for dealing with unbalanced classes. One popular one is SMOTE - Synthetic Minority Oversampling TEchnique - which creates new fake data that looks like it could reasonably be in the minority class. While it sounds risky to create fake data and use it to train your model, in many cases SMOTE, or a combination of SMOTE and undersampling, can be highly effective.

You can read more about under- and over-sampling methods <a style="color:#4d61f4" href="https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/">here</a>, and specifically about SMOTE <a style="color:#4d61f4" href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">here</a>. Both links include Python code for implementing them, using special libraries. If you are able to install those libraries, those links could be very helpful for tackling unbalanced classification problems; if you can't install those libraries because of your company's IT policies, the method shown in this notebook uses only standard libraries.</div>